In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "pRH2mTv7OkWGD4A96CVNeEybW"
    consumer_secret = "0oDEzI5c3WaSs0fmTdir3EEYQ9krqPuFXiL6Fs5AaVr6smleHU"
    access_token = "1514376686011265026-jQLzScQddyFxURV802yBJCWZUXOXB6"
    access_token_secret = "x7uCPpMWwpOVADQjCu7T6DQgUTnaHDZYNjri7giOP4VXs"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517640211483676673, 'username': 'JasonMinyard3', 'created_at': 'Fri Apr 22 23:03:17 +0000 2022', 'lng': -86.2664375, 'lat': 43.310683, 'text': '@MisakiShizumi @EricDJuly Absolutely!'}
{'id': 1517640212314300419, 'username': 'KayLoca', 'created_at': 'Fri Apr 22 23:03:17 +0000 2022', 'lng': -79.85736349999999, 'lat': 36.090396999999996, 'text': 'We as women shouldn’t have to work on our period 😭😭😭'}
{'id': 1517640212372930561, 'username': 'Deborah70546540', 'created_at': 'Fri Apr 22 23:03:17 +0000 2022', 'lng': -77.0143985, 'lat': 38.898602999999994, 'text': 'It is sad to read news articles and view on television news\xa0 on April 21, 2022\xa0 at 4:30pm there were 3 conservative… https://t.co/chAQSsCQ83'}
{'id': 1517640213568409605, 'username': 'KAunkst7', 'created_at': 'Fri Apr 22 23:03:17 +0000 2022', 'lng': -78.6531315, 'lat': 41.970092, 'text': "@fertile_lover Think you'll ever cut it?"}
{'id': 1517640212293246976, 'username': 'OHLBarrieColts', 'created_at': 'Fri Apr 22 23:03

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
